In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [2]:
%reload_ext autoreload
%autoreload 2

In [ ]:
path = Path("/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/wangcDrugRepoProject/BindDBdata/All_BindingDB/ESM2_BindingDB_random")
df = pd.read_parquet(path / "train.parquet", engine='fastparquet')
df.head()

In [ ]:
path = "/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/drug_discovery/output/models/Random_Forest_ESMv1_BindingDB_random_predictions.csv"
pred_df = pd.read_csv(Path(path))
pred_df.rename(columns={"drug_smile":"Drug", "affinity": "affinity_pred"}, inplace=True)
pred_df.head()

In [ ]:
df = pd.read_parquet(Path(path), )

In [5]:
import pyarrow.parquet as pq
path = Path("../output/data/combined_predictions_BindingDB.parquet")
df = pd.read_parquet(path)
# # Memory efficient approach for large files
# parquet_file = pq.ParquetFile(path)
# first_batch = next(parquet_file.iter_batches(batch_size=1000))
# df = first_batch.to_pandas()


In [6]:
df.head()

,row_index,Drug,BindingDB Ligand Name,Target Name,Target,Mutant,Kd (nM),Affinity,EC number,Target Class,...,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6,Drug_Features,Target_Features,Model,pred_affinity,Dataset,Split mode,Embedding
0,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",GBM,5.712360,BindingDB,random,ESMv1
1,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",XGBoost,5.183766,BindingDB,random,ESMv1
2,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",GBM,4.998585,BindingDB,random,ESMv1
3,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",XGBoost,4.958881,BindingDB,random,ESMv1
4,2,CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,"US11248001, Example 467",Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Mutant,200.0,6.698970,3.4.21.-,Enzyme,...,Serine protease SB clan,Serine protease S8A subfamily,None,"[0.7361834645271301, 0.798686683177948, 0.7603...","[-0.02277339994907379, 0.23574493825435638, -0...",GBM,5.742586,BindingDB,random,ESMv1


In [7]:
df["Embedding"].unique()

array(['ESMv1', 'ESM2', 'MUTAPLM', 'ProteinCLIP'], dtype=object)

In [8]:

df["Split mode"].unique()

array(['random', 'cold_protein', 'cold_drug'], dtype=object)

In [10]:
df["Model"].unique()

array(['GBM', 'XGBoost'], dtype=object)

In [ ]:
# Import the calculate_metrics function
import sys
sys.path.append('../src')
from ml_benchmark_util import calculate_metrics

# Extract true and predicted values
y_true = df['Affinity'].values
y_pred = df['affinity_pred'].values

# Calculate metrics
rmse, mae, mse, r2, pearson_corr, median_ae, explained_variance = calculate_metrics(y_true, y_pred)

# Display results
print(f"Metrics for {len(df)} samples:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")
print(f"Pearson Correlation: {pearson_corr:.4f}")
print(f"Median Absolute Error: {median_ae:.4f}")
print(f"Explained Variance: {explained_variance:.4f}")

In [ ]:
# Calculate metrics grouped by model and embedding type
results = []

for (embedding, model_name), group in df.groupby(['Embedding', 'model_name']):
    y_true = group['Affinity'].values
    y_pred = group['affinity_pred'].values
    
    rmse, mae, mse, r2, pearson_corr, median_ae, explained_variance = calculate_metrics(y_true, y_pred)
    
    results.append({
        'Embedding': embedding,
        'Model': model_name,
        'N_samples': len(group),
        'RMSE': rmse,
        'MAE': mae,
        'MSE': mse,
        'R2': r2,
        'Pearson': pearson_corr,
        'Median_AE': median_ae,
        'Explained_Variance': explained_variance
    })

# Convert to DataFrame for better display
metrics_df = pd.DataFrame(results)
print("Metrics by Embedding and Model:")
print(metrics_df.round(4))

In [ ]:
# Create a simple scatter plot of predictions vs true values
plt.figure(figsize=(10, 8))

# Plot by embedding type
embeddings = df['Embedding'].unique()
colors = plt.cm.Set1(np.linspace(0, 1, len(embeddings)))

for i, embedding in enumerate(embeddings):
    mask = df['Embedding'] == embedding
    plt.scatter(df[mask]['Affinity'], df[mask]['affinity_pred'], 
               alpha=0.6, label=embedding, color=colors[i], s=20)

# Add perfect prediction line
min_val = min(df['Affinity'].min(), df['affinity_pred'].min())
max_val = max(df['Affinity'].max(), df['affinity_pred'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.8, label='Perfect Prediction')

plt.xlabel('True Affinity', fontsize=12)
plt.ylabel('Predicted Affinity', fontsize=12)
plt.title('Predicted vs True Affinity Values (1000 samples)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()